<a href="https://colab.research.google.com/github/albertaillet/LELSD/blob/colab-setup/notebooks/train_lelsd_stylegan2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*   Clone repo and from correct branch
*   Download the pretrained stylegan2 model from https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/
*   Download the pretrained segmentation model from https://github.com/zllrunning/face-parsing.PyTorch





In [1]:
!git clone --branch colab-setup https://github.com/albertaillet/CLIPLSD/
!wget https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl -P LELSD/pretrained/stylegan2
!gdown --id 154JgKpzCPW82qINcVieuPH3fZ2e0P812 -O LELSD/pretrained/face_bisenet/model.pth

Cloning into 'LELSD'...
remote: Enumerating objects: 172, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 172 (delta 13), reused 0 (delta 0), pack-reused 139
Receiving objects: 100% (172/172), 29.26 MiB | 11.78 MiB/s, done.
Resolving deltas: 100% (32/32), done.
--2022-03-31 07:10:06--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 13.226.122.107, 13.226.122.47, 13.226.122.102, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|13.226.122.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381624121 (364M) [binary/octet-stream]
Saving to: ‘LELSD/pretrained/stylegan2/ffhq.pkl’

ffhq.pkl            100%[===================>] 363.94M  28.5MB/s    in 16s     

2022-03-31 07:10:22 (23.1 MB/s) - ‘LELSD/pretrained/stylegan2/ffhq.pkl’ saved [381624121/381624121]

Downloading...
From: https://

In [2]:
!pip install -r LELSD/requirements.txt

     |████████████████████████████████| 96 kB 3.1 MB/s 
     |████████████████████████████████| 3.0 MB 25.6 MB/s 
     |████████████████████████████████| 28.5 MB 2.0 MB/s 
     |████████████████████████████████| 831.4 MB 6.9 kB/s 
     |████████████████████████████████| 62 kB 736 kB/s 
     |████████████████████████████████| 76 kB 4.2 MB/s 
     |████████████████████████████████| 108 kB 37.7 MB/s 
     |████████████████████████████████| 10.3 MB 20.9 MB/s 
     |████████████████████████████████| 3.3 MB 37.6 MB/s 
     |████████████████████████████████| 2.9 MB 31.9 MB/s 
     |████████████████████████████████| 204 kB 50.7 MB/s 
     |████████████████████████████████| 2.4 MB 33.4 MB/s 
     |████████████████████████████████| 26.9 MB 1.6 MB/s 
     |████████████████████████████████| 195 kB 24.0 MB/s 
     |████████████████████████████████| 10.1 MB 41.0 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 164 kB 47.0 MB/s 
     |█████████████

In [1]:
%cd LELSD

/content/LELSD


In [3]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")
import sys
import torch

import models
from utils.stylegan2_utils import StyleGAN2SampleGenerator
from utils.segmentation_utils import FaceSegmentation, StuffSegmentation, GANLinearSegmentation
from lelsd import LELSD

# Training StyleGAN2 with Supervised Segmentation

### StyleGAN2 FFHQ

In [4]:
device = torch.device('cuda')

exp_dir = "out"
G2 = models.get_model("stylegan2", "pretrained/stylegan2/ffhq.pkl")
stylegan2_sample_generator = StyleGAN2SampleGenerator(G=G2, device=device)

face_bisenet = models.get_model("face_bisenet", "pretrained/face_bisenet/model.pth")
face_segmentation = FaceSegmentation(face_bisenet=face_bisenet, device=device)

latent_space = "Z"
loss_function = "L2"
mask_aggregation = 'average'
num_latent_dirs = 1
part_name = "mouth"
face_parts = ["mouth", "u_lip", "l_lip"]
lr = 0.001
min_alpha_value = -1.0
max_alpha_value = 1.0
min_abs_alpha_value = 0.0
gamma_correlation = 5.0
onehot_temperature = 0.001
batch_size = 1
localization_layers = list(range(1, 18))
localization_layer_weights = None
log_dir = f'{exp_dir}/lelsd_stylegan2_ffhq/{latent_space}_{loss_function}_{mask_aggregation}/{num_latent_dirs}D/face_bisenet/{part_name}'
lelsd = LELSD(device=device,
              localization_layers=localization_layers,
              semantic_parts=face_parts,
              loss_function=loss_function,
              localization_layer_weights=localization_layer_weights,
              mode='foreground',
              mask_aggregation=mask_aggregation,
              n_layers=18,
              latent_dim=512,
              num_latent_dirs=num_latent_dirs,
              learning_rate=lr,
              batch_size=batch_size,
              gamma_correlation=gamma_correlation,
              unit_norm=False,
              latent_space=latent_space,
              onehot_temperature=onehot_temperature,
              min_alpha_value=min_alpha_value,
              max_alpha_value=max_alpha_value,
              min_abs_alpha_value=min_abs_alpha_value,
              log_dir=log_dir,
              )
lelsd.fit(stylegan2_sample_generator, face_segmentation, num_batches=200 * num_latent_dirs,
          num_lr_halvings=3,
          pgbar=True, summary=True)
lelsd.save()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

  0%|          | 0/200 [00:00<?, ?it/s]

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


100%|██████████| 200/200 [04:43<00:00,  1.42s/it]


In [5]:
device = torch.device('cuda')

# exp_dir = "out"
# G2 = models.get_model("stylegan2", "pretrained/stylegan2/ffhq.pkl")
# stylegan2_sample_generator = StyleGAN2SampleGenerator(G=G2, device=device)

# face_bisenet = models.get_model("face_bisenet", "pretrained/face_bisenet/model.pth")
# face_segmentation = FaceSegmentation(face_bisenet=face_bisenet, device=device)

for latent_space in ["Z", "W", "W+"]:
    for loss_function in ["L2"]:
        for mask_aggregation in [
            'average',
            'union',
            'intersection',
        ]:

            for num_latent_dirs in [1, 2]:
                for part_name, face_parts in zip(
                        [
                            "mouth",
                            "skin",
                            "eyes",
                            "nose",
                            "ears",
                            "background",
                            "eyebrows",
                            "hair",
                            "cloth", "eyeglass"

                        ],
                        [
                            ["mouth", "u_lip", "l_lip"],
                            ["skin"],
                            ["l_eye", "r_eye"],
                            ["nose"],
                            ["l_ear", "r_ear", "earrings"],
                            ["background"],
                            ["l_brow", "r_brow"],
                            ["hair", "hat"],
                            ["hair"],
                            ["cloth", "neck", "necklace"],
                            ["eyeglass"]

                        ]
                ):
                    lr = 0.001
                    min_alpha_value = -1.0
                    max_alpha_value = 1.0
                    min_abs_alpha_value = 0.0
                    gamma_correlation = 5.0
                    onehot_temperature = 0.001
                    batch_size = 2
                    localization_layers = list(range(1, 18))
                    localization_layer_weights = None
                    log_dir = f'{exp_dir}/lelsd_stylegan2_ffhq/{latent_space}_{loss_function}_{mask_aggregation}/{num_latent_dirs}D/face_bisenet/{part_name}'
                    lelsd = LELSD(device=device,
                                  localization_layers=localization_layers,
                                  semantic_parts=face_parts,
                                  loss_function=loss_function,
                                  localization_layer_weights=localization_layer_weights,
                                  mode='foreground',
                                  mask_aggregation=mask_aggregation,
                                  n_layers=18,
                                  latent_dim=512,
                                  num_latent_dirs=num_latent_dirs,
                                  learning_rate=lr,
                                  batch_size=batch_size,
                                  gamma_correlation=gamma_correlation,
                                  unit_norm=False,
                                  latent_space=latent_space,
                                  onehot_temperature=onehot_temperature,
                                  min_alpha_value=min_alpha_value,
                                  max_alpha_value=max_alpha_value,
                                  min_abs_alpha_value=min_abs_alpha_value,
                                  log_dir=log_dir,
                                  )

                    lelsd.fit(stylegan2_sample_generator, face_segmentation, num_batches=200 * num_latent_dirs,
                              num_lr_halvings=3,
                              pgbar=True, summary=True)
                    lelsd.save()


 34%|███▍      | 69/200 [01:52<03:32,  1.62s/it]


KeyboardInterrupt: ignored

In [8]:
!ls

apps		 images    models     pretrained   requirements.txt
Dockerfile	 lelsd.py  notebooks  __pycache__  utils
environment.yml  LICENSE   out	      readme.md


In [9]:
!zip -r out/file.zip out/lelsd_stylegan2_ffhq/Z_L2_average

  adding: out/lelsd_stylegan2_ffhq/Z_L2_average/ (stored 0%)
  adding: out/lelsd_stylegan2_ffhq/Z_L2_average/1D/ (stored 0%)
  adding: out/lelsd_stylegan2_ffhq/Z_L2_average/1D/face_bisenet/ (stored 0%)
  adding: out/lelsd_stylegan2_ffhq/Z_L2_average/1D/face_bisenet/eyebrows/ (stored 0%)
  adding: out/lelsd_stylegan2_ffhq/Z_L2_average/1D/face_bisenet/eyebrows/Mar31_07-59-45/ (stored 0%)
  adding: out/lelsd_stylegan2_ffhq/Z_L2_average/1D/face_bisenet/eyebrows/Mar31_07-59-45/events.out.tfevents.1648713585.b8d3d969ece2.269.7 (deflated 70%)
  adding: out/lelsd_stylegan2_ffhq/Z_L2_average/1D/face_bisenet/eyebrows/Mar31_07-59-45/config.json (deflated 70%)
  adding: out/lelsd_stylegan2_ffhq/Z_L2_average/1D/face_bisenet/eyebrows/Mar31_07-59-45/batch=0-image_alpha=-0.62.jpg (deflated 8%)
  adding: out/lelsd_stylegan2_ffhq/Z_L2_average/1D/face_bisenet/eyebrows/Mar31_07-59-45/model.pth (deflated 34%)
  adding: out/lelsd_stylegan2_ffhq/Z_L2_average/1D/face_bisenet/mouth/ (stored 0%)
  adding: out/l

In [10]:
from google.colab import files
files.download("out/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### StyleGAN2 LSUN Church

In [ ]:
device = torch.device('cuda')

exp_dir = "../out"
G2 = models.get_model("stylegan2", "../pretrained/stylegan2/stylegan2-church-config-f.pkl")
stylegan2_sample_generator = StyleGAN2SampleGenerator(G=G2, device=device)

deeplabv2_resnet101 = models.get_model("cocostuff_deeplab",
                                       "../pretrained/cocostuff_deeplab/deeplabv2_resnet101_msc-cocostuff164k-100000.pth")
segmentation_model = StuffSegmentation(deeplabv2_resnet101=deeplabv2_resnet101, 
                                       config_path="../pretrained/cocostuff_deeplab/", device=device)

for latent_space in ["Z", "W", "W+"]:
    for loss_function in ["L2"]:
        for mask_aggregation in [
            'average',
            'union',
            'intersection',
        ]:

            for num_latent_dirs in [1, 2]:
                for part_name, sub_parts in zip(
                        [
                            "church",
                            "sky", "vegetation", "ground"

                        ],
                        [
                            ["building-other", "house"],
                            ["sky-other", "clouds"],
                            ["tree", "grass", "bush", "plant-other"],
                            ["dirt", "mud", "sand", "gravel", "ground-other", "road", "pavement"],

                        ]
                ):
                    lr = 0.001
                    min_alpha_value = -1.0
                    max_alpha_value = 1.0
                    min_abs_alpha_value = 0.0
                    gamma_correlation = 5.0
                    onehot_temperature = 0.001
                    batch_size = 4
                    localization_layers = list(range(1, 14))
                    localization_layer_weights = None
                    log_dir = f'{exp_dir}/lelsd_stylegan2_lsun_church/{latent_space}_{loss_function}_{mask_aggregation}/{num_latent_dirs}D/deeplab/{part_name}'
                    lelsd = LELSD(device=device,
                                  localization_layers=localization_layers,
                                  semantic_parts=sub_parts,
                                  loss_function=loss_function,
                                  localization_layer_weights=localization_layer_weights,
                                  mode='foreground',
                                  mask_aggregation=mask_aggregation,
                                  n_layers=14,
                                  latent_dim=512,
                                  num_latent_dirs=num_latent_dirs,
                                  learning_rate=lr,
                                  batch_size=batch_size,
                                  gamma_correlation=gamma_correlation,
                                  unit_norm=False,
                                  latent_space=latent_space,
                                  onehot_temperature=onehot_temperature,
                                  min_alpha_value=min_alpha_value,
                                  max_alpha_value=max_alpha_value,
                                  min_abs_alpha_value=min_abs_alpha_value,
                                  log_dir=log_dir,
                                  )

                    lelsd.fit(stylegan2_sample_generator, segmentation_model, num_batches=200 * num_latent_dirs,
                              num_lr_halvings=3,
                              pgbar=True, summary=True)
                    lelsd.save()


 13%|█▎        | 51/400 [00:51<05:30,  1.06it/s]

### StyleGAN2 LSUN Car

In [ ]:
device = torch.device('cuda')

exp_dir = "../out"
G2 = models.get_model("stylegan2", "../pretrained/stylegan2/stylegan2-car-config-f.pkl")
stylegan2_sample_generator = StyleGAN2SampleGenerator(G=G2, device=device)

deeplabv2_resnet101 = models.get_model("cocostuff_deeplab",
                                       "../pretrained/cocostuff_deeplab/deeplabv2_resnet101_msc-cocostuff164k-100000.pth")
segmentation_model = StuffSegmentation(deeplabv2_resnet101=deeplabv2_resnet101,
                                       config_path="../pretrained/cocostuff_deeplab/", device=device)

for latent_space in ["W", "W+"]:
    for loss_function in ["L2"]:
        for mask_aggregation in [
            'average',
        ]:
            for num_latent_dirs in [1, 2]:
                for part_name, sub_parts in zip(
                        [
                            "car",
                            "road", "sky", "grass+tree",

                        ],
                        [
                            ["car", "truck", "bus", "motorcycle"],
                            ["road", "pavement", "dirt"],
                            ["sky-other", "clouds"],
                            ["tree", "grass", "bush", "plant-other"],
                        ]
                ):
                    lr = 0.001
                    min_alpha_value = -1.0
                    max_alpha_value = 1.0
                    min_abs_alpha_value = 0.0
                    gamma_correlation = 5.0
                    onehot_temperature = 0.001
                    batch_size = 4
                    localization_layers = list(range(1, 16))
                    localization_layer_weights = None
                    log_dir = f'{exp_dir}/lelsd_stylegan2_lsun_car/{latent_space}_{loss_function}_{mask_aggregation}/{num_latent_dirs}D/deeplab/{part_name}'
                    lelsd = LELSD(device=device,
                                  localization_layers=localization_layers,
                                  semantic_parts=sub_parts,
                                  loss_function=loss_function,
                                  localization_layer_weights=localization_layer_weights,
                                  mode='foreground',
                                  mask_aggregation=mask_aggregation,
                                  n_layers=16,
                                  latent_dim=512,
                                  num_latent_dirs=num_latent_dirs,
                                  learning_rate=lr,
                                  batch_size=batch_size,
                                  gamma_correlation=gamma_correlation,
                                  unit_norm=False,
                                  latent_space=latent_space,
                                  onehot_temperature=onehot_temperature,
                                  min_alpha_value=min_alpha_value,
                                  max_alpha_value=max_alpha_value,
                                  min_abs_alpha_value=min_abs_alpha_value,
                                  log_dir=log_dir,
                                  )

                    lelsd.fit(stylegan2_sample_generator, segmentation_model, num_batches=200 * num_latent_dirs,
                              num_lr_halvings=3,
                              pgbar=True, summary=True)
                    lelsd.save()


100%|██████████| 400/400 [07:02<00:00,  1.06s/it]


### StyleGAN2 LSUN Horse

In [ ]:
device = torch.device('cuda')

exp_dir = "../out"
G2 = models.get_model("stylegan2", "../pretrained/stylegan2/stylegan2-horse-config-f.pkl")
stylegan2_sample_generator = StyleGAN2SampleGenerator(G=G2, device=device)

deeplabv2_resnet101 = models.get_model("cocostuff_deeplab",
                                       "../pretrained/cocostuff_deeplab/deeplabv2_resnet101_msc-cocostuff164k-100000.pth")
segmentation_model = StuffSegmentation(deeplabv2_resnet101=deeplabv2_resnet101,
                                       config_path="../pretrained/cocostuff_deeplab/", device=device)

for latent_space in ["W", "W+"]:
    for loss_function in ["L2"]:
        for mask_aggregation in [
            'average',
        ]:
            for num_latent_dirs in [1, 2]:
                for part_name, sub_parts in zip(
                        [
                            "horse",
                            "person", "sky", "grass+tree", "ground"

                        ],
                        [
                            ["horse"],
                            ["person"],
                            ["sky-other", "clouds"],
                            ["tree", "grass", "bush", "plant-other"],
                            ["dirt", "mud", "sand", "gravel", "ground-other", "road", "pavement"],

                        ]
                ):
                    lr = 0.001
                    min_alpha_value = -1.0
                    max_alpha_value = 1.0
                    min_abs_alpha_value = 0.0
                    gamma_correlation = 5.0
                    onehot_temperature = 0.001
                    batch_size = 4
                    localization_layers = list(range(1, 14))
                    localization_layer_weights = None
                    log_dir = f'{exp_dir}/lelsd_stylegan2_lsun_horse/{latent_space}_{loss_function}_{mask_aggregation}/{num_latent_dirs}D/deeplab/{part_name}'
                    lelsd = LELSD(device=device,
                                  localization_layers=localization_layers,
                                  semantic_parts=sub_parts,
                                  loss_function=loss_function,
                                  localization_layer_weights=localization_layer_weights,
                                  mode='foreground',
                                  mask_aggregation=mask_aggregation,
                                  n_layers=14,
                                  latent_dim=512,
                                  num_latent_dirs=num_latent_dirs,
                                  learning_rate=lr,
                                  batch_size=batch_size,
                                  gamma_correlation=gamma_correlation,
                                  unit_norm=False,
                                  latent_space=latent_space,
                                  onehot_temperature=onehot_temperature,
                                  min_alpha_value=min_alpha_value,
                                  max_alpha_value=max_alpha_value,
                                  min_abs_alpha_value=min_abs_alpha_value,
                                  log_dir=log_dir,
                                  )

                    lelsd.fit(stylegan2_sample_generator, segmentation_model, num_batches=200 * num_latent_dirs,
                              num_lr_halvings=3,
                              pgbar=True, summary=True)
                    lelsd.save()


 40%|████      | 80/200 [01:15<01:53,  1.06it/s]

### StyleGAN2 MetFaces

In [ ]:
device = torch.device('cuda')

exp_dir = "../out"
G2 = models.get_model("stylegan2", "../pretrained/stylegan2/metfaces.pkl")
stylegan2_sample_generator = StyleGAN2SampleGenerator(G=G2, device=device)

face_bisenet = models.get_model("face_bisenet", "../pretrained/face_bisenet/model.pth")
face_segmentation = FaceSegmentation(face_bisenet=face_bisenet, device=device)

for latent_space in ["W", "W+"]:
    for loss_function in ["L2"]:
        for mask_aggregation in [
            'average',
        ]:

            for num_latent_dirs in [1, 2]:
                for part_name, face_parts in zip(
                        [
                            "mouth",
                            "skin",
                            "eyes",
                            "nose",
                            "ears",
                            "background",
                            "eyebrows",
                            "hair",
                            "cloth",
                        ],
                        [
                            ["mouth", "u_lip", "l_lip"],
                            ["skin"],
                            ["l_eye", "r_eye"],
                            ["nose"],
                            ["l_ear", "r_ear", "earrings"],
                            ["background"],
                            ["l_brow", "r_brow"],
                            ["hair", "hat"],
                            ["hair"],
                            ["cloth", "neck", "necklace"],

                        ]
                ):
                    lr = 0.001
                    min_alpha_value = -1.0
                    max_alpha_value = 1.0
                    min_abs_alpha_value = 0.0
                    gamma_correlation = 5.0
                    onehot_temperature = 0.001
                    batch_size = 4
                    localization_layers = list(range(1, 18))
                    localization_layer_weights = None
                    log_dir = f'{exp_dir}/lelsd_stylegan2_metfaces/{latent_space}_{loss_function}_{mask_aggregation}/{num_latent_dirs}D/face_bisenet/{part_name}'
                    lelsd = LELSD(device=device,
                                  localization_layers=localization_layers,
                                  semantic_parts=face_parts,
                                  loss_function=loss_function,
                                  localization_layer_weights=localization_layer_weights,
                                  mode='foreground',
                                  mask_aggregation=mask_aggregation,
                                  n_layers=18,
                                  latent_dim=512,
                                  num_latent_dirs=num_latent_dirs,
                                  learning_rate=lr,
                                  batch_size=batch_size,
                                  gamma_correlation=gamma_correlation,
                                  unit_norm=False,
                                  latent_space=latent_space,
                                  onehot_temperature=onehot_temperature,
                                  min_alpha_value=min_alpha_value,
                                  max_alpha_value=max_alpha_value,
                                  min_abs_alpha_value=min_abs_alpha_value,
                                  log_dir=log_dir,
                                  )

                    lelsd.fit(stylegan2_sample_generator, face_segmentation, num_batches=200 * num_latent_dirs,
                              num_lr_halvings=3,
                              pgbar=True, summary=True)
                    lelsd.save()
